In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<user>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['CPI_RegionUrban_raw',
 'LaborForce_clean',
 'employment_prediction',
 'population_clean',
 'GDP_monthly',
 'population_predicted_2024',
 'employment_clean',
 'AvgWeeklyEarningsAllEmp_raw',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'msa_codes',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'arima_unem_pred_score',
 'population_raw',
 'Population_monthly',
 'Employment_clean',
 'LaborForce_raw',
 'Employment_raw',
 'unemployment_raw',
 'GDP_clean',
 'UnemploymentRate_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
df = pd.read_csv('employment_MSA.csv')
print(df.dtypes)
df

GeoFips     int64
GeoName    object
2001        int64
2002        int64
2003        int64
2004        int64
2005        int64
2006        int64
2007        int64
2008        int64
2009        int64
2010        int64
2011        int64
2012        int64
2013        int64
2014        int64
2015        int64
2016        int64
2017        int64
2018        int64
2019        int64
dtype: object


,GeoFips,GeoName,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,"Abilene, TX (Metropolitan Statistical Area)",89511,90338,91132,91998,92937,94097,96484,99264,...,96809,98193,99843,101199,102316,102611,102601,102786,104536,107405
1,10420,"Akron, OH (Metropolitan Statistical Area)",396954,396663,396391,404591,412097,417595,424012,424079,...,402554,407811,412049,414502,421225,426047,427394,428034,430261,433800
2,10500,"Albany, GA (Metropolitan Statistical Area)",78011,77100,78282,78126,80177,80621,81153,80663,...,77793,79189,79388,79694,80577,81418,81714,81872,82431,83286
3,10540,"Albany-Lebanon, OR (Metropolitan Statistical A...",54749,54136,53710,54697,57088,58162,59546,59473,...,55562,55341,55676,56161,57670,58997,59661,61777,63422,64277
4,10580,"Albany-Schenectady-Troy, NY (Metropolitan Stat...",516432,514437,518106,524002,530740,534257,540904,543929,...,528095,531541,537634,543162,548362,556276,563056,569909,575772,579135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,"Yakima, WA (Metropolitan Statistical Area)",112685,112734,114189,113223,114927,117779,118536,121722,...,119123,120041,124847,125387,127305,128600,130601,132743,134982,136556
380,49620,"York-Hanover, PA (Metropolitan Statistical Area)",211514,211138,211658,217006,223488,226836,231790,232142,...,221987,224717,225936,227264,229563,231735,235643,237721,240494,243152
381,49660,"Youngstown-Warren-Boardman, OH-PA (Metropolita...",311150,305452,302591,301803,303520,302814,301433,295953,...,280982,285225,287302,287503,287915,287937,288202,284315,284853,283700
382,49700,"Yuba City, CA (Metropolitan Statistical Area)",61499,62404,62573,63898,64648,66167,67736,65442,...,62526,62563,63966,65465,66816,68645,69903,71388,73096,75581


In [4]:
df.drop(columns='GeoName', inplace=True)
df

,GeoFips,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,96809,98193,99843,101199,102316,102611,102601,102786,104536,107405
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,402554,407811,412049,414502,421225,426047,427394,428034,430261,433800
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,77793,79189,79388,79694,80577,81418,81714,81872,82431,83286
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,55562,55341,55676,56161,57670,58997,59661,61777,63422,64277
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,528095,531541,537634,543162,548362,556276,563056,569909,575772,579135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,112685,112734,114189,113223,114927,117779,118536,121722,120007,119123,120041,124847,125387,127305,128600,130601,132743,134982,136556
380,49620,211514,211138,211658,217006,223488,226836,231790,232142,223008,221987,224717,225936,227264,229563,231735,235643,237721,240494,243152
381,49660,311150,305452,302591,301803,303520,302814,301433,295953,280869,280982,285225,287302,287503,287915,287937,288202,284315,284853,283700
382,49700,61499,62404,62573,63898,64648,66167,67736,65442,63426,62526,62563,63966,65465,66816,68645,69903,71388,73096,75581


In [5]:
df.rename(columns={'GeoFips':'CBSA'}, inplace=True)
df

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,96809,98193,99843,101199,102316,102611,102601,102786,104536,107405
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,402554,407811,412049,414502,421225,426047,427394,428034,430261,433800
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,77793,79189,79388,79694,80577,81418,81714,81872,82431,83286
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,55562,55341,55676,56161,57670,58997,59661,61777,63422,64277
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,528095,531541,537634,543162,548362,556276,563056,569909,575772,579135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,112685,112734,114189,113223,114927,117779,118536,121722,120007,119123,120041,124847,125387,127305,128600,130601,132743,134982,136556
380,49620,211514,211138,211658,217006,223488,226836,231790,232142,223008,221987,224717,225936,227264,229563,231735,235643,237721,240494,243152
381,49660,311150,305452,302591,301803,303520,302814,301433,295953,280869,280982,285225,287302,287503,287915,287937,288202,284315,284853,283700
382,49700,61499,62404,62573,63898,64648,66167,67736,65442,63426,62526,62563,63966,65465,66816,68645,69903,71388,73096,75581


In [6]:
# for loop to predict 2024 values
predictions = []
for i in range(0,384):
    y = df.iloc[i, 1:].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    predictions.append(pred.values.tolist()[-1])
df['2024'] = predictions

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: Convergen

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [7]:
df

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,...,98193,99843,101199,102316,102611,102601,102786,104536,107405,112292.651309
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,...,407811,412049,414502,421225,426047,427394,428034,430261,433800,433800.450867
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,...,79189,79388,79694,80577,81418,81714,81872,82431,83286,83288.410862
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,...,55341,55676,56161,57670,58997,59661,61777,63422,64277,66996.410264
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,...,531541,537634,543162,548362,556276,563056,569909,575772,579135,579041.672221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,112685,112734,114189,113223,114927,117779,118536,121722,120007,...,120041,124847,125387,127305,128600,130601,132743,134982,136556,143425.087120
380,49620,211514,211138,211658,217006,223488,226836,231790,232142,223008,...,224717,225936,227264,229563,231735,235643,237721,240494,243152,243150.572092
381,49660,311150,305452,302591,301803,303520,302814,301433,295953,280869,...,285225,287302,287503,287915,287937,288202,284315,284853,283700,281284.153105
382,49700,61499,62404,62573,63898,64648,66167,67736,65442,63426,...,62563,63966,65465,66816,68645,69903,71388,73096,75581,79268.288705


In [33]:
df.sort_values(by='2024').head(10)

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
139,24500,47844,47693,47643,48326,48759,49919,50132,50810,50231,...,49736,49761,50030,50082,50307,50161,50320,50590,51202,0.000000
88,19180,40683,40062,40636,40029,40190,39863,39718,39183,37825,...,38010,38165,37345,37667,37770,36640,36004,35462,35539,35520.500431
363,47460,31490,31744,32161,32019,31930,32237,32619,33951,33663,...,33717,34107,34752,35334,35106,35509,35689,36390,36827,36942.630348
201,30300,33539,33446,33600,33585,33984,34467,35176,34975,33927,...,33980,33494,34437,34882,35302,35917,36445,36124,36318,37060.128562
110,21420,33369,33265,33999,34452,34825,35569,36830,38102,37704,...,38074,39161,39620,40232,39839,39709,37503,37222,37423,37251.384712
57,16180,41635,41771,41908,43058,43878,43921,42266,41157,39407,...,38076,37406,37531,37521,37670,38260,39425,39484,40470,40679.919199
322,42700,32571,34134,35970,36151,37340,38629,40082,38869,37758,...,37769,38022,37520,38221,38834,39586,40561,40788,41266,41398.260849
138,24420,33417,33916,35230,36699,37991,38788,38856,37739,35858,...,35008,34959,35030,36111,37117,37764,38765,39779,40997,41763.896643
271,38220,49489,48574,48948,49671,49426,49191,48352,47892,47573,...,47240,46585,45148,43947,43835,43550,42950,42830,42396,42329.966920
155,25980,34716,34713,35794,37376,37293,37608,38874,41570,41165,...,44250,43430,43178,42013,41526,40817,42036,42816,43145,43667.921438


In [34]:
df.drop([139], inplace=True)

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,...,98193,99843,101199,102316,102611,102601,102786,104536,107405,112292.651309
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,...,407811,412049,414502,421225,426047,427394,428034,430261,433800,433800.450867
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,...,79189,79388,79694,80577,81418,81714,81872,82431,83286,83288.410862
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,...,55341,55676,56161,57670,58997,59661,61777,63422,64277,66996.410264
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,...,531541,537634,543162,548362,556276,563056,569909,575772,579135,579041.672221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,112685,112734,114189,113223,114927,117779,118536,121722,120007,...,120041,124847,125387,127305,128600,130601,132743,134982,136556,143425.087120
380,49620,211514,211138,211658,217006,223488,226836,231790,232142,223008,...,224717,225936,227264,229563,231735,235643,237721,240494,243152,243150.572092
381,49660,311150,305452,302591,301803,303520,302814,301433,295953,280869,...,285225,287302,287503,287915,287937,288202,284315,284853,283700,281284.153105
382,49700,61499,62404,62573,63898,64648,66167,67736,65442,63426,...,62563,63966,65465,66816,68645,69903,71388,73096,75581,79268.288705


In [35]:
df['2024'] = df['2024'].astype('int')
df.head()

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,...,98193,99843,101199,102316,102611,102601,102786,104536,107405,112292
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,...,407811,412049,414502,421225,426047,427394,428034,430261,433800,433800
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,...,79189,79388,79694,80577,81418,81714,81872,82431,83286,83288
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,...,55341,55676,56161,57670,58997,59661,61777,63422,64277,66996
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,...,531541,537634,543162,548362,556276,563056,569909,575772,579135,579041


In [36]:
df['2024_Emp_ROC'] = (df['2024']-df['2019'])/df['2019'] *100
df.head()

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2024,2024_Emp_ROC
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,...,99843,101199,102316,102611,102601,102786,104536,107405,112292,4.550068
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,...,412049,414502,421225,426047,427394,428034,430261,433800,433800,0.000000
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,...,79388,79694,80577,81418,81714,81872,82431,83286,83288,0.002401
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,...,55676,56161,57670,58997,59661,61777,63422,64277,66996,4.230129
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,...,537634,543162,548362,556276,563056,569909,575772,579135,579041,-0.016231


In [38]:
df_pred = df[['CBSA', '2024_Emp_ROC']].copy()
df_pred

,CBSA,2024_Emp_ROC
0,10180,4.550068
1,10420,0.000000
2,10500,0.002401
3,10540,4.230129
4,10580,-0.016231
...,...,...
379,49420,5.030171
380,49620,-0.000823
381,49660,-0.851604
382,49700,4.878210


In [39]:
df_pred['Emp_Score'] = df_pred['2024_Emp_ROC'].rank(ascending=True)

In [42]:
df_pred.head()

,CBSA,2024_Emp_ROC,Emp_Score
0,10180,4.550068,270.0
1,10420,0.000000,93.5
2,10500,0.002401,97.0
3,10540,4.230129,263.0
4,10580,-0.016231,61.0


In [43]:
# create new collection in mongo
arima_emp_pred_score = db.arima_emp_pred_score

In [44]:
df_dict = df_pred.to_dict(orient='records')

In [45]:
arima_emp_pred_score.insert_many(df_dict)